# DCP analysis

In this exercise, you will fix optimization problems that break the DCP rules by identifying the DCP error and then rewriting the problem.

In [ ]:
# Problem 1.
import cvxpy as cp

x = cp.Variable()
cost = (x**2 + 1) ** 0.5
prob = cp.Problem(cp.Minimize(cost))
# prob.solve()

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
# sqrt is a concave function and the inner argument is neither increasing nor affine
# a way to rewrite this is to use norm2


cost = cp.norm(cp.hstack([x, 1]), 2)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()


In [ ]:
# Problem 2.
import cvxpy as cp

x = cp.Variable()
prob = cp.Problem(cp.Minimize(x + 2), [5 == 2 * cp.inv_pos(x)])

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
# when a constraint has an equal sign both sides must be affine but in 
# this case the right side is convex so multiplying by x on both sides 
# works fine
prob = cp.Problem(cp.Minimize(x + 2), [5 * x == 2])
prob.solve(), x.value

In [ ]:
# Problem 3.
import cvxpy as cp

x = cp.Variable()
prob = cp.Problem(cp.Minimize(x + 2), [5 <= 2 * x**-2])

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
# the right side of the constraint is undefined on zero so multiplying
# by x^2 on both sides works fine

prob = cp.Problem(cp.Minimize(x + 2), [5 * x**2 <= 2])
prob.solve(), x.value

In [ ]:
# Problem 4.
import cvxpy as cp

x = cp.Variable()
y = cp.Variable()
prob = cp.Problem(cp.Minimize(cp.pos(x)), [0 <= cp.quad_over_lin(x, y), y >= 1])

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
prob = cp.Problem(cp.Minimize(cp.pos(x)), [0 >= cp.quad_over_lin(x, y), y >= 1])
# prob = cp.Problem(cp.Minimize(cp.pos(x)), [0 >= cp.power(x,2)])
prob.solve(), x.value, y.value


In [ ]:
# Problem 5.
import cvxpy as cp

x = cp.Variable()
prob = cp.Problem(cp.Minimize(x + 2), [cp.exp(2 * x) + cp.exp(3 * x) <= cp.exp(5 * x)])

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
# convex functions on boths sides of inequality, by appling log to both sides left 
# side remains convex and right side is now affine
prob = cp.Problem(cp.Minimize(x + 2), [cp.log_sum_exp(cp.hstack([2 * x, 3 * x])) <= 5 * x])
prob.solve()


In [ ]:
# Bonus problem. Do this for extra credit.
import cvxpy as cp
import numpy as np

x = cp.Variable()
prob = cp.Problem(cp.Maximize(-((cp.maximum(x, 4) - 3) ** 2)), [x >= 1])

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
# The sign is unknown after subtracting 3, so pushing that 
# inside the maximum guarantees that is positive
prob = cp.Problem(cp.Maximize(-((cp.maximum(x - 3, 1)) ** 2)), [x >= 1])
prob.solve(), x.value

In [ ]:
# Bonus problem. Do this for extra credit.
# This is a real problem from the CVXPY forum.
import cvxpy as cp
import numpy as np

m = 10
np.random.seed(1)
c = np.random.randn(m, 1)
c = np.abs(c)  # Important: This is nonnegative.
u = np.random.randn(m, 1)
u = np.abs(u)  # Important: This is nonnegative.

x = cp.Variable(m)
cost = sum([c[i] * x[i] * cp.inv_pos(u[i] - x[i]) for i in range(m)])

# TODO explain why the problem isn't DCP and rewrite it to satisfy DCP.
# DCP only handles multiplications of a function with a constant so multiplying x and 1/x won't work
# by doing some math it's possible to reorder the terms and have a single x

# cost = sum([-c[i] * ( 1 + u[i] * cp.inv_pos(x[i] - u[i])) for i in range(m)])
cost = sum([c[i] * (u[i] * cp.inv_pos(u[i] - x[i]) - 1) for i in range(m)])

prob = cp.Problem(cp.Minimize(cost))
prob.solve(), x.value